In [532]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import ast
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from IPython.display import Markdown
from collections import Counter
from fonctions import *

In [238]:
from sklearn.feature_extraction.text import CountVectorizer

In [163]:
def contains_only_false(lst):
    return all(not val for val in lst)

In [164]:
def delete_items_with_prefix(dictionnaire, prefix):
    keys_to_delete = [key for key in dictionnaire if key.startswith(prefix)]
    for key in keys_to_delete:
        del dictionnaire[key]
#30,34,38,42,43,45,54

In [165]:
def filter_sentences_based_on_boolean(sentences_dict, boolean_dict):
    filtered_dict = {}
    for key, sentences in sentences_dict.items():
        boolean_values = boolean_dict.get(key)
        if boolean_values is not None and len(sentences) == len(boolean_values):
            filtered_sentences = [sentence for sentence, boolean_value in zip(sentences, boolean_values) if boolean_value]
            filtered_dict[key] = filtered_sentences
    return filtered_dict

In [166]:
os.chdir('/home/noe/Documents/2022-2023/ENC/Mémoire/dictionnaires_lemmes/')

In [167]:
file = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/PDV_grams.xlsx'
viard = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/Viard_grams.xlsx'

In [168]:
df_pdv = pd.read_excel(file)
df_viard = pd.read_excel(viard)

In [169]:
df = pd.DataFrame({'texts':df_pdv['textes'],'clean_PDV': df_pdv['textes'].apply(clean_text)})

pdv = {}
for i in range(len(df)):
    for sentence in df['clean_PDV'][i]:
        pdv[f'pdv_{i+1}_{df["clean_PDV"][i].index(sentence)+1}'] = sentence

In [170]:
df2 = pd.DataFrame({'viard_texts':df_viard['text'],'clean_viard':df_viard['text'].apply(clean_text)})

column_without_brackets = [[string.replace('[', '').replace(']', '').lstrip().rstrip() for string in sublist] for sublist in df2['clean_viard']]

df2['clean'] = column_without_brackets
viard = {}
for i in range(len(df2)):
    for sentence in df2['clean'][i]:        
        viard[f'viard_{i+1}_{df2["clean"][i].index(sentence)+1}'] = sentence

In [447]:
texte_viard = ' '.join(list(viard_stopwords.values()))

In [446]:
viard_stopwords

{'viard_1_1': 'philippus ualesii andegauie comes francie nauarre regna regens notum facimus uniuersis presentibus futuris quod cum stephanus le haubergier ciuis parisiensis quoddam nobile feodum ualoris duodecim librarum parisiensium annui redditus uel circa situm uilla antogniaco eius territorio quod religioso uiro abbate sancti germani pratis prope parisius tenetur homagium certo legitimo titulo sicut asserit duxerit acquirendum prefatus tamen abbas eumdem stephanum eo quod nobilis non existit suum recipere homagium memorato feodo recusauit adhuc recu sat unde nobis prefatus stephanus humiliter supplicauit ut super hoc sibi specialem gratiam facere dignaremur',
 'viard_1_2': 'nos igitur eiusdem hac parte supplicationibus fauorabiliter annuentes eidem stephano speciali gratia concedimus se heredibus suis ut ipse ipsius heredes licet idem stephanus a nobilibus originem non traxerit dictum feodum si esset nobilis tenere possint imperpetuum pacifice quiete ipsumque stephanum auctoritate 

In [171]:
delete_items_with_prefix(viard,'viard_30_')
delete_items_with_prefix(viard,'viard_34_')
delete_items_with_prefix(viard,'viard_38_')
delete_items_with_prefix(viard,'viard_42_')
delete_items_with_prefix(viard,'viard_43_')
delete_items_with_prefix(viard,'viard_45_')
delete_items_with_prefix(viard,'viard_54_')
delete_items_with_prefix(viard,'viard_64_')
delete_items_with_prefix(viard,'viard_65_')

In [172]:
pdv_liste_values = list(pdv.values())
pdv_liste_keys = list(pdv.keys())

print(len(pdv_liste_keys))
print(len(remove_words_from_sentence_list(pdv_liste_values)))

viard_liste_values = list(viard.values())
viard_liste_keys = list(viard.keys())

2074
2074


In [12]:
#list_stop_words = ['et','de','per','pro','ad','ac','in','at']

In [350]:
pdv_stopwords = dict(zip(pdv_liste_keys,remove_words_from_sentence_list(pdv_liste_values)))
viard_stopwords = dict(zip(viard_liste_keys,remove_words_from_sentence_list(viard_liste_values)))

In [351]:
def is_single_word(s):
    return isinstance(s, str) and len(s.split()) == 1

def keys_with_single_word_value(dictionary):
    single_word_keys = []
    for key, value in dictionary.items():
        if is_single_word(value):
            single_word_keys.append(key)
    return single_word_keys

In [352]:
unique_values = keys_with_single_word_value(viard_stopwords)

In [353]:
for i in range(len(unique_values)):
    keys = list(viard_stopwords.keys())
    clé = keys.index(unique_values[i])
    viard_stopwords[keys[clé-1]] =  viard_stopwords.get(keys[clé-1]) + ' ' + viard_stopwords.get(unique_values[i])
    del viard_stopwords[keys[clé]]

In [239]:
def have_similar_bigram(sentence1, sentence2):
    # Create a CountVectorizer for bigrams
    vectorizer = CountVectorizer(ngram_range=(2, 2))

    # Extract bigrams and compute the bigram vectors
    bigram_matrix = vectorizer.fit_transform([sentence1,sentence2])

    # Convert the sentences to bigram vectors
    bigram_vector1 = bigram_matrix[0].toarray()
    bigram_vector2 = bigram_matrix[1].toarray()

    # Check if there is at least one common bigram
    has_common_bigram = (bigram_vector1 & bigram_vector2).any()

    return has_common_bigram

In [ ]:
idx = []

for element in tqdm(range(len(viard_stopwords))):
    
    try:
    
        phrase1 = list(viard_stopwords.values())[element]

        bigrams = []

        for i in range(len(pdv_stopwords)):

            phrase2 = list(pdv_stopwords.values())[i]

            x = have_similar_bigram(phrase1,phrase2)

            bigrams.append(x)

        han = []
        for i in range(len(bigrams)):
            if bigrams[i] == True:
                han.append(list(pdv_stopwords.keys())[i])

        idx.append(han)
    except Exception as e:
        print(f"An unexpected error occurred: {element}", e)
        idx.append([])

In [392]:
match_bi = {key:value for key,value in zip(list(viard_stopwords.keys()),idx)}

In [14]:
'''match_bi_word_without_stopwords = {}

for i in tqdm(list(pdv_stopwords.keys())):
    big = create_bigrams(pdv_stopwords.get(i))
    correspondance = []
    lesgrams = []

    for key in list(viard_stopwords.keys()):
        liste_bigrams = create_bigrams(viard_stopwords.get(key))
        for bigram in big:
            if bigram in liste_bigrams:
                correspondance.append(key)
                lesgrams.append(bigram)
                break
                
    match_bi_word_without_stopwords[i] = correspondance
delete_empty_values(match_bi_word_without_stopwords)'''

100%|███████████████████████████████████████| 2074/2074 [00:39<00:00, 52.27it/s]


In [394]:
viard_words_stopwords = []

for idx in list(match_bi.keys()):
    viard_words_stopwords.append(viard_stopwords.get(idx))
    
pdv_words_stopwords = []

for liste in list(match_bi.values()):
    sentence = []
    for value in liste:
        sentence.append(pdv_stopwords.get(value))
    pdv_words_stopwords.append(sentence)
    
sentences_stopwords = dict(zip(viard_words_stopwords,pdv_words_stopwords))

In [484]:
def find_first_and_last_indices(text, sequence):
    words = text.split()
    sequence_words = sequence.split()
    start_index = None
    end_index = None

    for i in range(len(words) - len(sequence_words) + 1):
        if words[i:i+len(sequence_words)] == sequence_words:
            start_index = i
            end_index = i + len(sequence_words) - 1
            break

    return start_index, end_index+1

In [503]:
def find_first_and_last_common_word_indices(sentence1, sentence2):
    words1 = sentence1.split()
    words2 = sentence2.split()
    common_words = set(words1) & set(words2)

    first_common_index = None
    last_common_index = None

    for idx, word in enumerate(words1):
        if word in common_words:
            first_common_index = idx
            break

    for idx, word in reversed(list(enumerate(words1))):
        if word in common_words:
            last_common_index = idx
            break

    return first_common_index, last_common_index

In [492]:
start_index, end_index = find_first_and_last_indices(texte_viard, viard_stopwords.get('viard_1_1'))
start_index,end_index

(0, 87)

In [493]:
' '.join(texte_viard.split()[start_index:end_index])

'philippus ualesii andegauie comes francie nauarre regna regens notum facimus uniuersis presentibus futuris quod cum stephanus le haubergier ciuis parisiensis quoddam nobile feodum ualoris duodecim librarum parisiensium annui redditus uel circa situm uilla antogniaco eius territorio quod religioso uiro abbate sancti germani pratis prope parisius tenetur homagium certo legitimo titulo sicut asserit duxerit acquirendum prefatus tamen abbas eumdem stephanum eo quod nobilis non existit suum recipere homagium memorato feodo recusauit adhuc recu sat unde nobis prefatus stephanus humiliter supplicauit ut super hoc sibi specialem gratiam facere dignaremur'

In [505]:
viard_stopwords.get('viard_1_1').split()[13]

'quod'

In [504]:
find_first_and_last_common_word_indices(viard_stopwords.get('viard_1_1'),pdv_stopwords.get('pdv_3_12'))

(13, 62)

In [502]:
print(len(pdv_stopwords.get('pdv_3_12').split()))
pdv_stopwords.get('pdv_3_12')

50


'contumaciam nostram magistri thadei suessa magnae curiae nostrae iudicis fidelis nostri plene procurationis mandatum excusat cuius auctoritas eo quod fuerat criminale iudicium quod frustra procurator interuenit nullatenus eneruatur cum ipsius citationis tenor quae nos aut procuratores nostros exegit manifeste contrarium dedisset intelligi uidelicet quod contra nos non criminaliter fuisset agendum'

## autre partie

In [396]:
useless_words = ['ut','sic','non']

In [397]:
liste_bool_fin = []

for i in range(len(sentences_stopwords)):
    
    item = list(sentences_stopwords.items())[i]
    
    liste_bool = []

    phrase1 = item[0].split()


    for element in range(len(item[1])):

        phrase2 = (item[1][element]).split()

        true_false = [word in phrase2 for word in phrase1]

        count_boolean = true_false.count(True)

        if count_boolean >= 3:
            liste_bool.append(True)
        else:
            true_false = False
            liste_bool.append(False)
    liste_bool_fin.append(liste_bool)


In [398]:
three = dict(zip(list(sentences_stopwords),liste_bool_fin))
startos = {key: value for key, value in three.items() if not contains_only_false(value)}

In [399]:
liste_keys = []

for elm in list(startos.keys()):
    liste_keys.append(get_key(elm,pdv_stopwords))
    
liste_values = []

for i in liste_keys:
    liste_values.append(match_bi_word_without_stopwords.get(i))
    
my_dict = dict(zip(liste_keys, liste_values))

In [20]:
values_sentences = list(sentences_stopwords.values())
values_idx = list(three.values())

In [52]:
for key,value in pdv_stopwords.items():
    if value == '©':
        del pdv_stopwords[key]
        print(key,value)
#x[114]

In [400]:
x = list(pdv_stopwords.values())
y = list(viard_stopwords.values())

In [54]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)

liste_dic_pdv = []
for i in tqdm(range(len(x))):
    
    try:
        sentences: List[str] = [x[i]]
        # Get the main object from the model (: data iterator + postprocesor
        from pie_extended.models.lasla.imports import get_iterator_and_processor
        for sentence_group in sentences:
            iterator, processor = get_iterator_and_processor()
            dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
            liste_dic_pdv.append(dic)
    except Exception as e:
        print(f"An unexpected error occurred: {i}", e)

100%|███████████████████████████████████████| 2051/2051 [06:31<00:00,  5.24it/s]


In [55]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)

liste_dic_viard = []
for i in tqdm(range(len(y))):
    
    try:
        sentences: List[str] = [y[i]]
        # Get the main object from the model (: data iterator + postprocesor
        from pie_extended.models.lasla.imports import get_iterator_and_processor
        for sentence_group in sentences:
            iterator, processor = get_iterator_and_processor()
            dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
            liste_dic_viard.append(dic)
    except Exception as e:
        print(f"An unexpected error occurred: {i}", e)

100%|█████████████████████████████████████████| 705/705 [02:21<00:00,  5.00it/s]


In [401]:
liste_finale_pdv = []

for i in tqdm(liste_dic_pdv):
    liste_sentence = []
    for j in i:
        liste_sentence.append(j.get('lemma'))
    liste_finale_pdv.append(liste_sentence)

liste_finale_viard = []

for i in tqdm(liste_dic_viard):
    liste_sentence = []
    for j in i:
        liste_sentence.append(j.get('lemma'))
    liste_finale_viard.append(liste_sentence)

100%|██████████████████████████████████████| 705/705 [00:00<00:00, 52321.19it/s]


In [402]:
liste_finale_pdv = [' '.join(sublist) for sublist in liste_finale_pdv]
liste_finale_viard = [' '.join(sublist) for sublist in liste_finale_viard]

pdv_lemma = {key: value for key, value in zip(list(pdv_stopwords.keys()), liste_finale_pdv)}
viard_lemma = {key: value for key, value in zip(list(viard_stopwords.keys()), liste_finale_viard)}

In [76]:
def count_keys_startswith(dct, prefix):
    count = 0
    for key in dct:
        if key.startswith(prefix):
            count += 1
    return count

In [403]:
liste_last =[]

for i in range(1,706):
    nbr_cles = list(range(1,count_keys_startswith(viard_lemma,f'viard_{i}_')+1))
    liste_last.append(nbr_cles)
flattened_list = [num for sublist in liste_last for num in sublist]
string_list = list(map(str, flattened_list))

In [404]:
liste_cles_viard = list(viard_lemma.keys())

In [405]:
new_list = []
for idx in range(len(viard_lemma)):
    new_str = liste_cles_viard[idx].split('_')[0]+'_'+liste_cles_viard[idx].split('_')[1] + '_' + string_list[idx]
    
    new_list.append(new_str)

viard_2 = dict(zip(new_list,list(viard_lemma.values())))

In [406]:
viard2_liste_values = list(viard_2.values())
viard2_liste_keys = list(viard_2.keys())
viard2_stopwords = dict(zip(viard2_liste_keys,remove_words_from_sentence_list(viard2_liste_values)))

{'viard_1_1': 'philippus Ualesius andegauis comes francies nauo regnum rego notus2 facio uniuersus preso futurus qui2 cum3 stephanus lego? haurio ciuis parisiensis quidam nobilis2 feudus ualor duodecim libra parisiensis annuus reddo uel2 circa2 situs2 uilla antogniacus is territorium qui1 religiosus uir abbate sanctus germanus2 pratum prope1 parisius teneo homagium certus legitimus titulus sicut1 assero? duco acquiro prefor tamen abba idem stephanus eo3 qui2 nobilis2 non exsisto suus recipio homagius memoro feodus recuso adhuc recus satis2 unde1 nos prefor stephanus humiliter supplico ut4 super2 hic1 sui1 specialis gratia facio dignor',
 'viard_1_2': 'nos igitur idem hic1 pars supplicatio fauorabiliter annuo idem stephanus specialis gratia concedo1 sui1 heres suus ut4 ipse ipse heres licet1 idem stephanus ab nobilis1 origo non traho dictum feudus si2 sum1 nobilis2 teneo possum1 imperpetuus pacifice quies ipse que stephanus auctoritas noster habilito qui3 hic1 preso uolens precipio qui2

## TFIDF

In [407]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [415]:
def compare_sentences(sentence1, sentence2):

    vectorizer = TfidfVectorizer()

    # Compute TF-IDF scores for the sentences
    tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2])

    # Convert the sentences to TF-IDF vectors
    tfidf_vector1 = tfidf_matrix[0]
    tfidf_vector2 = tfidf_matrix[1]

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tfidf_vector1, tfidf_vector2)

    return cosine_sim[0][0]

In [529]:
phrase1 = viard_stopwords.get('viard_1_1')
phrase2 = pdv_stopwords.get('pdv_1_2')

In [530]:
print(phrase1)
print('-----------')
print(phrase2)

philippus ualesii andegauie comes francie nauarre regna regens notum facimus uniuersis presentibus futuris quod cum stephanus le haubergier ciuis parisiensis quoddam nobile feodum ualoris duodecim librarum parisiensium annui redditus uel circa situm uilla antogniaco eius territorio quod religioso uiro abbate sancti germani pratis prope parisius tenetur homagium certo legitimo titulo sicut asserit duxerit acquirendum prefatus tamen abbas eumdem stephanum eo quod nobilis non existit suum recipere homagium memorato feodo recusauit adhuc recu sat unde nobis prefatus stephanus humiliter supplicauit ut super hoc sibi specialem gratiam facere dignaremur
-----------
quid facimus inquiunt quia hic homo hostibus sic triumphat si sic ipsum dimittimus omnem sibi subiciet gloriam lombardorum more cesareo ueniens non tardabit ut posse nobis locum auferat destruat gentem nostram


In [531]:
compare_sentences(phrase1,phrase2)

0.04662554188259434

In [422]:
print(len(pdv_lemma.get('pdv_1_6').split()))
print(len(viard_lemma.get('viard_1_1').split()))

44
87


## inverted 

In [205]:
test = {}

for i in tqdm(list(viard2_stopwords.keys())):
    big = create_bigrams(viard2_stopwords.get(i))
    correspondance = []

    for key in list(pdv_stopwords.keys()):
        liste_bigrams = create_bigrams(pdv_stopwords.get(key))
        for bigram in big:
            if bigram in liste_bigrams:
                correspondance.append(key)
                lesgrams.append(bigram)
                
    
    test[i] = correspondance
delete_empty_values(test)

100%|█████████████████████████████████████████| 705/705 [00:40<00:00, 17.60it/s]


In [179]:
viard_words_stopwords = []

for idx in list(test.keys()):
    viard_words_stopwords.append(viard2_stopwords.get(idx))
    
pdv_words_stopwords = []

for liste in list(test.values()):
    sentence = []
    for value in liste:
        sentence.append(pdv_stopwords.get(value))
    pdv_words_stopwords.append(sentence)
    
test2 = dict(zip(viard_words_stopwords,pdv_words_stopwords))

In [181]:
test3 = []

for i in range(len(test2)):
    
    item = list(test2.items())[i]
    
    liste_bool = []

    phrase1 = item[0].split()


    for element in range(len(item[1])):

        phrase2 = (item[1][element]).split()

        true_false = [word in phrase2 for word in phrase1]

        count_boolean = true_false.count(True)

        if count_boolean >= 3:
            liste_bool.append(True)
        else:
            true_false = False
            liste_bool.append(False)
    test3.append(liste_bool)

In [190]:
flat = [item for sublist in list(test2.values()) for item in sublist]

{'viard_1_1': 'philippus ualesii andegauie comes francie nauarre regna regens notum facimus uniuersis presentibus futuris quod cum stephanus le haubergier ciuis parisiensis quoddam nobile feodum ualoris duodecim librarum parisiensium annui redditus uel circa situm uilla antogniaco eius territorio quod religioso uiro abbate sancti germani pratis prope parisius tenetur homagium certo legitimo titulo sicut asserit duxerit acquirendum prefatus tamen abbas eumdem stephanum eo quod nobilis non existit suum recipere homagium memorato feodo recusauit adhuc recu sat unde nobis prefatus stephanus humiliter supplicauit ut super hoc sibi specialem gratiam facere dignaremur',
 'viard_1_2': 'nos igitur eiusdem hac parte supplicationibus fauorabiliter annuentes eidem stephano speciali gratia concedimus se heredibus suis ut ipse ipsius heredes licet idem stephanus a nobilibus originem non traxerit dictum feodum si esset nobilis tenere possint imperpetuum pacifice quiete ipsumque stephanum auctoritate 

In [543]:
match_bi

{'viard_1_1': ['pdv_3_12',
  'pdv_3_16',
  'pdv_8_4',
  'pdv_21_3',
  'pdv_21_23',
  'pdv_21_25',
  'pdv_21_26',
  'pdv_21_41',
  'pdv_21_48',
  'pdv_22_9',
  'pdv_24_16',
  'pdv_28_11',
  'pdv_44_2',
  'pdv_47_5',
  'pdv_62_5',
  'pdv_69_7',
  'pdv_69_8',
  'pdv_73_4',
  'pdv_74_5',
  'pdv_76_1',
  'pdv_81_3',
  'pdv_85_1',
  'pdv_107_3',
  'pdv_115_4',
  'pdv_116_7',
  'pdv_118_8',
  'pdv_121_1',
  'pdv_137_11',
  'pdv_145_2',
  'pdv_147_1',
  'pdv_149_1',
  'pdv_153_3',
  'pdv_154_9',
  'pdv_156_5',
  'pdv_158_3',
  'pdv_161_14',
  'pdv_200_3',
  'pdv_204_2',
  'pdv_210_2',
  'pdv_214_2',
  'pdv_216_1',
  'pdv_216_4',
  'pdv_218_4',
  'pdv_218_5',
  'pdv_219_1',
  'pdv_219_2',
  'pdv_219_7',
  'pdv_222_3',
  'pdv_227_2',
  'pdv_230_2',
  'pdv_230_3',
  'pdv_233_1',
  'pdv_234_1',
  'pdv_235_1',
  'pdv_235_2',
  'pdv_236_1',
  'pdv_236_3',
  'pdv_240_2',
  'pdv_240_4',
  'pdv_240_5',
  'pdv_241_1',
  'pdv_241_2',
  'pdv_243_2',
  'pdv_244_2',
  'pdv_245_2',
  'pdv_249_1',
  'pdv_250_